In [14]:
import requests

# request headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
    'Accept-Encoding': 'utf-8'
}

In [11]:
def recurrent_area_search(areas: list):
    id_area = []
    for area in areas:
        id_area.append((area['id'], area['name']))
        if 'areas' in area.keys():
            if len(area['areas']) > 0:
                id_area.extend(recurrent_area_search(area['areas']))
    return id_area

# getting area names and their codes in HH appropriate format
area_codes = requests.get('https://api.hh.ru/areas').json()
area_codes = {a.lower(): i for i, a in recurrent_area_search(area_codes)}
list(area_codes.items())[:5]

[('россия', '113'),
 ('республика марий эл', '1620'),
 ('виловатово', '4228'),
 ('волжск', '1621'),
 ('звенигово', '1622')]

In [90]:
from urllib import parse
import urllib3
from bs4 import BeautifulSoup
import time
import random

def lookup_resumes(query: str, location: str, page: int, english_only: bool = False):
    """ 
        Function for looking up candidates on hh.ru

        query: str
            - Search query that is contained in candidates' resume description
        location: str
            - Location name in lowercase (e.g. 'москва') for searching for candidates
        page: int
            - Page number for the search. Each page contains 20 resumes
        english_only: bool
            - View resumes in English only. Some candidates have not made resumes in English so some fields are going to
            left empty in case of this flag set to True

        returns:
            list of resume ids as strings
    """

    http = urllib3.PoolManager()
    urlencoded_query = parse.quote(query, safe='')

    # forming a url for accessing page with resumes
    lookup_url = f'https://hh.ru/search/resume?exp_period=all_time&L_lenient=true&clusters=true&pos=full_text&area={area_codes[location]}&text={urlencoded_query}&page={page}'
    if english_only:
        lookup_url += '&logic=normal'
    
    # requesting page with resumes with given query
    resp = http.request(
        method='GET',
        url=lookup_url,
        headers=headers
    )

    # searching for resumes on HTML page
    soup = BeautifulSoup(resp.data.decode('utf-8', 'ignore'), "html.parser")
    resumes = soup.findAll("div", {"data-resume-id" : True})
    resume_ids = [r.find('a')['href'].split('?')[0].split('/')[2] for r in resumes]

    return resume_ids

In [91]:
# getting 100 candidates 
candidate_ids = []

# NOTE: if english_only flag set to True then the latin search query is going to work better. 
# Otherwise type query in Russian.
for i in range(5):
    candidate_ids.extend(lookup_resumes('data', 'москва', 10))
    time.sleep(random.uniform(0,3))

In [80]:
import re


def process_column_data(columns, first_key: str, second_key: str):
    """ 
        Function that converts BeautifulSoup object containing HTML with rows of two columns into 
        dictionary with desired first key for the value in the first column and second key for the
        value in the second column

        columns
            - BeautifulSoup object containing data with rows of columns
        first_key: str
            - Key for the value in the first column
        second_key: str
            - Key for the value in the second column

        returns:
            List (each element represents a row) of dictionaries (each represents columns in the row)
    """

    if columns is not None:
        wo_tags = [re.sub(r'<[^>]*>', '', str(el)) for el in columns.find_all('div', {'class': 'bloko-column'})[1:]]
        return [{first_key: re.sub('\\xa0', ' ', el), second_key: re.sub('\\xa0', ' ', wo_tags[i+1 if i%2==0 else i+2])} for i, el in enumerate(wo_tags[::2])]
    return []

def get_resume_details(resume_id: str):
    """ 
        Returns prettified details on candidate's resume

        resume_id: str
            - Candidate's resume ID, that we get from lookup_resumes() 

        returns:
            Dictionary with information on the candidate
    """

    http = urllib3.PoolManager()
    lookup_url = f'https://hh.ru/resume/{resume_id}'
    resp = http.request(
        method='GET',
        url=lookup_url,
        headers=headers
    )

    # getting resume's information
    soup = BeautifulSoup(resp.data.decode('utf-8', 'ignore'), "html.parser")
    resume = soup.find('div', {'class': 'resume-applicant'})

    while resume is None:
        time.sleep(random.uniform(0,3))
        return get_resume_details(resume_id)
    
    agesoup = resume.find('span', {'data-qa': 'resume-personal-age'})
    sexsoup = resume.find('span', {'data-qa': 'resume-personal-gender'})
    areasoup = resume.find('span', {'data-qa': 'resume-personal-address'})
    dlicensesoup = resume.find('div', {'data-qa': 'resume-block-driver-experience'})
    jobpossoup = resume.find('li', {'data-qa': 'resume-block-position-specialization'})
    skillssoup = resume.find('div', {'data-qa': 'skills-table'})
    aboutsoup = resume.find('div', {'data-qa': 'resume-block-skills-content'})
    langsoup = resume.find('div', {'data-qa': 'resume-block-languages'})
    extrasoup = resume.find('div', {'data-qa': 'resume-block-additional'})

    return {
        'age': re.findall(r'\d+', agesoup.text)[0] if agesoup is not None else None,
        'sex': sexsoup.text if sexsoup is not None else None,
        'area': areasoup.text if areasoup is not None else None,
        'd_licence': re.sub('\\xa0', ' ', dlicensesoup.find_all('div',{'class': 'resume-block-container'})[1].text) \
            if dlicensesoup is not None else None,
        'job_positions': jobpossoup.text if jobpossoup is not None else None,
        'skills': [el.text for el in skillssoup.find_all('span')[1:]] if skillssoup is not None else None,
        'about': aboutsoup.text if aboutsoup is not None else None,
        'experience': process_column_data(resume.find('div', {'data-qa': 'resume-block-experience'}),'time','place'),
        'education': process_column_data(resume.find('div', {'data-qa': 'resume-block-education'}),'time','institute'),
        'languages': [el.text for el in langsoup.find_all('p')] if langsoup is not None else None,
        'courses': process_column_data(resume.find('div', {'data-qa': 'resume-block-additional-education'}),'time','course'), 
        'extra': [el.text for el in extrasoup.find_all('p')] if extrasoup is not None else None
    }

In [92]:
candidates_details = [get_resume_details(id) for id in candidate_ids]

In [93]:
import pandas as pd

candidates_df = pd.DataFrame(candidates_details)
candidates_df.head()

,age,sex,area,d_licence,job_positions,skills,about,experience,education,languages,courses,extra
0,33,Женщина,Москва,None,Переводчик,"[Пользователь ПК, Ответственность, Стрессоусто...",Увлекаюсь чтением книг в частности классики. Я...,[{'time': 'Январь 2021 — по настоящее время2 г...,"[{'time': '2013', 'institute': 'Международный ...","[Русский — Родной, Английский — C1 — Продвинут...","[{'time': '2013', 'course': '对外经贸大学Hsk 六级, 商务英...","[Гражданство: Россия, Разрешение на работу: Ро..."
1,34,Женщина,Москва,Имеется собственный автомобильПрава категории B,Специалист службы безопасности,"[Электронный документооборот, Контроль исполне...","Личные качества: стрессоустойчивость, исполнит...",[{'time': 'Февраль 2023 — по настоящее время7 ...,"[{'time': '2011', 'institute': 'Балтийская гос...","[Русский — Родной, Английский — A1 — Начальный]","[{'time': '2022', 'course': 'Институт развития...","[Гражданство: Россия, Желательное время в пути..."
2,37,Мужчина,Москва,Имеется собственный автомобильПрава категории B,Другое,"[Взыскание задолженности, Суды общей юрисдикци...",Взаимодействие с ФССП.\nВозможна работа по сов...,[{'time': 'Март 2015 — по настоящее время8 лет...,"[{'time': '2016', 'institute': 'Московский фин...","[Русский — Родной, Английский — A1 — Начальный]",[],"[Гражданство: Россия, Разрешение на работу: Ро..."
3,49,Женщина,Москва,None,Лаборант,[Исполнительность],None,[],"[{'time': '1996', 'institute': 'Калужский госу...",[Русский — Родной],[],"[Гражданство: Россия, Разрешение на работу: Ро..."
4,35,Мужчина,Москва,Права категории B,Менеджер по работе с партнерами,"[Управление затратами, Управление персоналом, ...",Готов рассматривать разные сферы деятельности....,[{'time': 'Декабрь 2020 — Август 20221 год 9 м...,"[{'time': '2009', 'institute': 'Московский гос...","[Русский — Родной, Английский — B1 — Средний]",[],"[Гражданство: Россия, Разрешение на работу: Ро..."
